# 3D

### Define problem, initial and boundary conditions

In [1]:

import time
import wandb
import pickle
import numpy as np
import tensorflow as tf
from math import pi, exp
from scipy.stats import norm
import matplotlib.pyplot as plt
from wandb.keras import WandbCallback

n_dim = 3

#Fix seeds
random_seed = 29
np.random.seed(random_seed)
tf.random.set_seed(random_seed)


F:\anaconda3\envs\PINN\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:

class Sampler_IBC():
    def __init__(self, lb, ub, cond=None, N_points=100,
                 method='sobol', grid=None, split=False, DTYPE='float64'):
        self.lb = lb
        self.ub = ub
        self.cond = cond
        self.DTYPE = DTYPE
        self.sample(N_points, method, grid, split)

    def sample(self, N_points, method, grid, split):
        if method == 'uniform':
            x_ibc = np.random.uniform(0, 1, size=(N_points, self.ub.shape[0]))
            x_ibc = self.lb + (self.ub - self.lb)*x_ibc
        elif method == 'latin':
            from pyDOE import lhs
            x_ibc = self.lb + (self.ub - self.lb)*lhs(self.ub.shape[0],N_points)
        elif method == 'sobol':
            import sobol
            x_ibc = sobol.sample(dimension=self.ub.shape[0], n_points=N_points)
            x_ibc = self.lb + (self.ub - self.lb)*x_ibc
        elif method == 'equi':
            x_ibc = np.linspace(self.lb, self.ub, N_points)
        elif method == 'grid':
            x_ibc = np.linspace(self.lb, self.ub, N_points).T
            temp_final = list()
            for val in x_ibc[0]:
                temp_final.append( [val] )
            dim = 1
            while dim < x_ibc.shape[0]:
                temp = list()
                for t1 in range(x_ibc.shape[1]):
                    for t2 in range(len(temp_final)):
                        temp_val = temp_final[t2].copy()
                        temp_val.append( x_ibc[dim, t1] )
                        temp.append( temp_val )
                temp_final = temp
                dim += 1
            x_ibc = np.array(temp_final)
        elif method == 'grid_old':
            idx = np.random.choice(range(grid.shape[0]),N_points,replace=False)
            x_ibc = grid[idx]
        if self.cond != None:
            y_ibc = self.cond(x_ibc)
            self.y = tf.cast(tf.Variable(y_ibc, trainable=False ),
                             self.DTYPE)
        if split:
            x_ibc, t_ibc = x_ibc[:, :-1], x_ibc[:, -1:]
            self.t = tf.cast(tf.Variable(t_ibc, trainable=False ),
                             self.DTYPE)
        self.x = tf.cast(tf.Variable(x_ibc, trainable=False ),
                         self.DTYPE)


In [3]:

#Define global variables
r = 0.01
T = 3
K = 10
sigma = [[0.05, 0.01, 0.1], [0.01, 0.06, -0.03], [0.1, -0.03, 0.4]]
sigma = np.array(sigma)
if np.linalg.det(sigma) != 0:
    if np.allclose(sigma, sigma.T):
        if np.all(np.linalg.eigvals(sigma) > 0):
            print('No problem with Covariance Matrix')
        else:
            print('Matrix is not positive semidefinite')
    else:
        print('Matrix is not symmetric')
else:
    print('Matrix is singular')

d = np.array([r]*n_dim) - np.array([0]*n_dim)
alphas = np.zeros((n_dim, n_dim))
for i in range(n_dim):
    alphas[i, i] = np.sum( np.dot(sigma[i], sigma[i]) )
    for j in range(i):
        alphas[i, j] = alphas[j, i] = np.sum( np.dot(sigma[i], sigma[j]) )
DTYPE = 'float32'

#Define domain boundaries
lb = np.array( ([0.]*n_dim)+[0] )
ub = np.array( ([3.*K]*n_dim)+[T] )


No problem with Covariance Matrix


In [4]:

#Problem Definition
def h_2(inp):
    res = - np.ones( inp.shape[0] )
    return res

def g(inp):
    res = list()
    for inp_val in inp:
        res.append( np.max([0,
                            K*np.exp(-r*(T-inp_val[-1]))-\
                            np.min(inp_val[:-1])]) )
    return np.array(res)

def u_0(inp):
    res = list()
    for inp_val in inp:
        res.append( np.max([0, K - np.min(inp_val[:-1])]) )
    return np.array(res)

def s_0(inp):
    res = np.ones( (inp.shape[0], n_dim) ) * K
    return np.array(res)

#Point sampling
N_to_sample = 9000

#---------------- PDE Conditions
print('PDE COnditions\n')
#Initial
init_sampler = Sampler_IBC(np.array( ([0.]*n_dim)+[T] ),
                           np.array( ([3.*K]*n_dim)+[T] ),
                           u_0, 4000, DTYPE=DTYPE )
print( f'x: {init_sampler.x.shape}     y: {init_sampler.y.shape}' )

#Dirichlet
lb_dir = ([0.]*n_dim)+[0]; lb_dir[0] = 3.*K
dir_sampler = Sampler_IBC(np.array( lb_dir ),
                          np.array( ([3.*K]*n_dim)+[T] ),
                          g, N_to_sample//n_dim, DTYPE=DTYPE )
for dim in range(1, n_dim):
    lb_dir = ([0.]*n_dim)+[0]; lb_dir[dim] = 3.*K
    temp_sampler = Sampler_IBC(np.array( lb_dir ),
                              np.array( ([3.*K]*n_dim)+[T] ),
                              g, N_to_sample//n_dim, DTYPE=DTYPE )
    dir_sampler.x = tf.concat([dir_sampler.x, temp_sampler.x], axis=0)
    dir_sampler.y = tf.concat([dir_sampler.y, temp_sampler.y], axis=0)
print( f'x: {dir_sampler.x.shape}     y: {dir_sampler.y.shape}' )

#Neumann
print('No')

#---------------- Free Boundary Conditions
print('\nFree Boundary COnditions\n')
#Initial
init_fb_sampler = Sampler_IBC(np.array([T]), np.array([T]),
                              s_0, 1, DTYPE=DTYPE )
print( f't: {init_fb_sampler.x.shape}     y: {init_fb_sampler.y.shape}' )

#Dirichlet
dir_fb_sampler = Sampler_IBC(np.array([0.]), np.array([T]),
                             None, N_to_sample, DTYPE=DTYPE )
print( f't: {dir_fb_sampler.x.shape}' )

#Neumann
neu_fb_sampler = Sampler_IBC(np.array([0.]), np.array([T]),
                             None, N_to_sample, DTYPE=DTYPE )
print( f't: {neu_fb_sampler.x.shape}' )

#---------------- Conditions passed to PINN
pinn_conditions = {'Initial':init_sampler,
                   'Dirichlet':dir_sampler,
                   'Neumann':None}
fb_conditions = {'Initial':init_fb_sampler,
                 'Dirichlet':dir_fb_sampler,
                 'Neumann':neu_fb_sampler}


PDE COnditions

x: (4000, 4)     y: (4000,)
x: (9000, 4)     y: (9000,)
No

Free Boundary COnditions

t: (1, 1)     y: (1, 3)
t: (9000, 1)
t: (9000, 1)


In [5]:

test_sampler = Sampler_IBC(lb, ub, cond=None, DTYPE=DTYPE,
                           N_points=1000000, method='uniform', split=True)

#Point sampling
sample_to_test = 1000

#---------------- PDE Conditions
print('PDE COnditions\n')
#Initial
init_sampler_test = Sampler_IBC(np.array( ([0.]*n_dim)+[T] ),
                           np.array( ([3.*K]*n_dim)+[T] ),
                           u_0, sample_to_test, DTYPE=DTYPE, method='uniform' )
print( f'x: {init_sampler_test.x.shape}     y: {init_sampler_test.y.shape}' )

#Dirichlet
lb_dir = ([0.]*n_dim)+[0]; lb_dir[0] = 3.*K
dir_sampler_test = Sampler_IBC(np.array( lb_dir ),
                          np.array( ([3.*K]*n_dim)+[T] ),
                          g, sample_to_test//n_dim, DTYPE=DTYPE, method='uniform' )
for dim in range(1, n_dim):
    lb_dir = ([0.]*n_dim)+[0]; lb_dir[dim] = 3.*K
    temp_sampler = Sampler_IBC(np.array( lb_dir ),
                              np.array( ([3.*K]*n_dim)+[T] ),
                              g, sample_to_test//n_dim, DTYPE=DTYPE, method='uniform' )
    dir_sampler_test.x = tf.concat([dir_sampler_test.x, temp_sampler.x], axis=0)
    dir_sampler_test.y = tf.concat([dir_sampler_test.y, temp_sampler.y], axis=0)
print( f'x: {dir_sampler_test.x.shape}     y: {dir_sampler_test.y.shape}' )

#Neumann
print('No')

#---------------- Free Boundary Conditions
print('\nFree Boundary COnditions\n')
#Initial
init_fb_sampler_test = Sampler_IBC(np.array([T]), np.array([T]),
                              s_0, 1, DTYPE=DTYPE, method='uniform' )
print( f't: {init_fb_sampler_test.x.shape}     y: {init_fb_sampler_test.y.shape}' )

#Dirichlet
dir_fb_sampler_test = Sampler_IBC(np.array([0.]), np.array([T]),
                             None, sample_to_test, DTYPE=DTYPE, method='uniform' )
print( f't: {dir_fb_sampler_test.x.shape}' )

#Neumann
neu_fb_sampler_test = Sampler_IBC(np.array([0.]), np.array([T]),
                                  None, sample_to_test//n_dim, DTYPE=DTYPE, method='uniform' )
print( f't: {neu_fb_sampler_test.x.shape}' )

#---------------- Conditions passed to PINN
pinn_cond_test = {'Initial':init_sampler_test,
                   'Dirichlet':dir_sampler_test,
                   'Neumann':None}
fb_cond_test = {'Initial':init_fb_sampler_test,
                 'Dirichlet':dir_fb_sampler_test,
                 'Neumann':neu_fb_sampler_test}


PDE COnditions

x: (1000, 4)     y: (1000,)
x: (999, 4)     y: (999,)
No

Free Boundary COnditions

t: (1, 1)     y: (1, 3)
t: (1000, 1)
t: (333, 1)


In [6]:

#---------------- PDE Conditions
# initial
init_S_t = init_sampler.x.numpy()
init_P = init_sampler.y.numpy()
init_header = "1st underlying asset price, 2nd underlying asset price, 3rd underlying asset price, time, price"
init_data = np.hstack([init_S_t, init_P.reshape(-1, 1)])
np.savetxt(f"3D_init_data.csv", init_data, delimiter=", ", header=init_header)

# dirichlet
dir_S_t = dir_sampler.x.numpy()
dir_P = dir_sampler.y.numpy()
dir_header = "time, 1st underlying asset price, 2nd underlying asset price, 3rd underlying asset price, price"
dir_data = np.hstack([dir_S_t, dir_P.reshape(-1, 1)])
np.savetxt(f"3D_dir_data.csv", dir_data, delimiter=", ", header=dir_header)

#---------------- Free Boundary Conditions
# initial_fb
init_fb_S_t = init_fb_sampler.x.numpy()
init_fb_P = init_fb_sampler.y.numpy()
init_fb_header = "time, 1st underlying asset price, 2nd underlying asset price, 3rd underlying asset price"
init_fb_data = np.hstack([init_fb_S_t, init_fb_P])
np.savetxt(f"3D_init_fb_data.csv", init_fb_data, delimiter=", ", header=init_fb_header)

# dirichlet_fb
dir_fb_header = "underlying asset price"
dir_fb_data = dir_fb_sampler.x.numpy()
np.savetxt(f"3D_dir_fb_data.csv", dir_fb_data, delimiter=", ", header=dir_fb_header)

# neumann_fb
neu_fb_data = neu_fb_sampler.x.numpy()
neu_fb_header = "underlying asset price"
np.savetxt(f"3D_neu_fb_data.csv", neu_fb_data, delimiter=", ", header=neu_fb_header)

############## test ##############
#---------------- PDE Conditions
# the whole data
test_data = test_sampler.x.numpy()  #only underlying asset price, without time
test_header = "1st underlying asset price, 2nd underlying asset price, 3rd underlying asset price"
np.savetxt(f"3D_test_data.csv", test_data, delimiter=", ", header=test_header)

# initial_test
init_test_S_t = init_sampler_test.x.numpy()
init_test_P = init_sampler_test.y.numpy()
init_test_header = "1st underlying asset price, 2nd underlying asset price, 3rd underlying asset price, time, price"
init_test_data = np.hstack([init_test_S_t, init_test_P.reshape(-1, 1)])
np.savetxt(f"3D_test_init_data.csv", init_test_data, delimiter=", ", header=init_test_header)

# dirichlet_test
dir_test_S_t = dir_sampler_test.x.numpy()
dir_test_P = dir_sampler_test.y.numpy()
dir_test_header = "time, 1st underlying asset price, 2nd underlying asset price, 3rd underlying asset price, price"
dir_test_data = np.hstack([dir_test_S_t, dir_test_P.reshape(-1, 1)])
np.savetxt(f"3D_test_dir_data.csv", dir_test_data, delimiter=", ", header=dir_test_header)

#---------------- Free Boundary Conditions
# initial_fb_test
init_fb_test_S_t = init_fb_sampler_test.x.numpy()
init_fb_test_P = init_fb_sampler_test.y.numpy()
init_fb_test_header = "time, 1st underlying asset price, 2nd underlying asset price, 3rd underlying asset price"
init_fb_test_data = np.hstack([init_fb_test_S_t, init_fb_test_P])
np.savetxt(f"3D_test_init_fb_data.csv", init_fb_test_data, delimiter=", ", header=init_fb_header)

# dirichlet_fb_test
dir_fb_test_data = dir_fb_sampler_test.x.numpy()
dir_fb_test_header = "underlying asset price"
np.savetxt(f"3D_test_dir_fb_data.csv", dir_fb_test_data, delimiter=", ", header=dir_fb_header)

# neumann_fb_test
neu_fb_test_data = neu_fb_sampler_test.x.numpy()
neu_fb_test_header = "underlying asset price"
np.savetxt(f"3D_test_neu_fb_data.csv", neu_fb_test_data, delimiter=", ", header=neu_fb_test_header)


### PINN

In [7]:

#Define PDE
def pde(tape, vars, fun_val, fo_grads):
    second_order = 0
    first_order = 0
    zero_order = 0
    zero_order += tape.gradient(fun_val, vars[-1]) - r * fun_val
    for i, var in enumerate(vars[:-1]):
        dx_i = fo_grads[i]
        first_order += d[i] * var * dx_i
        second_order +=\
        tape.gradient(dx_i, var) * (var**2) * alphas[i, i]
        for j, var2 in enumerate(vars[:i]):
            second_order +=\
            2*( tape.gradient(dx_i, var2) * var * var2 * alphas[i, j])
    f = second_order/2 + first_order + zero_order
    del(tape)
    return f


In [8]:

import tensorflow as tf
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow import GradientTape as G_Tape
from tensorflow.keras.layers import Input, Dense, Reshape
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.initializers import GlorotNormal, GlorotUniform
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.metrics import mean_squared_error

class FreeBoundary_PINN():

    def __init__(self, params, pde, ibc_cond, ibc_fb_cond, lb, ub,
                 N_f=10000, N_fb_ibc=150, DTYPE='float64', coll_points=None):
        self.params = params
        self.DTYPE = DTYPE
        self.Default_Params()
        #Set seed
        if self.params['seed'] != None:
            tf.keras.utils.set_random_seed( self.params['seed'] )
            tf.config.experimental.enable_op_determinism()
        #Define pde
        self.pde = pde
        #Define intial and boundary conditions
        self.ibc_cond = ibc_cond
        self.ibc_fb_cond = ibc_fb_cond
        #All needed for points sampling
        self.lb = tf.Variable(lb, trainable=False)
        self.ub = tf.Variable(ub, trainable=False)
        if coll_points == None:
            self.N_f = N_f
            self.Sample_Points()
        else:
            self.x_f = coll_points[0]
            self.t_f = coll_points[1]
        self.N_fb_ibc = N_fb_ibc
        #Initialize the class: define the network
        self.Define_Regularizer()
        self.Define_Initializer()
        self.Define_Optimizer()
        self.Create_Network()
        self.Create_FB_Network()

    def Default_Params(self):
        target = self.params.keys()
        if 'seed' not in target:
            self.params['seed'] = None
        if 'optimizer' not in target:
            self.params['optimizer'] = 'Adam'
        if 'fb_optimizer' not in target:
            self.params['fb_optimizer'] = 'Adam'
        if 'reg_type' not in target:
            self.params['reg_type'] = None
        if 'initializer' not in target:
            self.params['initializer'] = 'glorot_normal'
        if 'activation' not in target:
            self.params['activation'] = 'tanh'
        if 'output_act' not in target:
            self.params['output_act'] = 'linear'
        if 'pde_weight' not in target:
            self.params['pde_weight'] = 1.
        if 'sup_weight' not in target:
            self.params['sup_weight'] = [1., 1., 1.]
        if 'fb_weight' not in target:
            self.params['fb_weight'] = [1., 1., 1.]
        if 'patience' not in target:
            self.params['patience'] = np.inf
        if 'sample_method' not in target:
            self.params['sample_method'] = 'uniform'
        if 'fb_output_act' not in target:
            self.params['fb_output_act'] = 'linear'
        if 'fb_activation' not in target:
            self.params['fb_activation'] = 'tanh'
        if 'verbose' not in target:
            self.params['verbose'] = 1
        if 'steps_fb_per_pde' not in target:
            self.params['steps_fb_per_pde'] = 1
        if 'fb_freezing' not in target:
            self.params['fb_freezing'] = None

    def Define_Regularizer(self):
        if self.params['reg_type'] == 'l1':
            self.regularizer = l1( self.params['reg'] )
        elif self.params['reg_type'] == 'l2':
            self.regularizer = l2( self.params['reg'] )
        elif self.params['reg_type'] == 'l1_l2':
            self.regularizer = l1_l2( self.params['reg'][0],
                                      self.params['reg'][1] )
        else:
            self.regularizer = None

    def Define_Initializer(self):
        if self.params['initializer'] == 'glorot_normal':
            self.initializer = GlorotNormal()
        elif self.params['initializer'] == 'glorot_uniform':
            self.initializer = GlorotUniform()
        else:
            self.initializer = None

    def Define_Optimizer(self):
        temp = self.params['optimizer']
        if temp.lower() == 'adam':
            self.opt = Adam( self.params['lr'] )
        elif temp.lower() == 'rmsprop':
            self.opt = RMSprop( self.params['lr'] )
        else:
            raise ValueError(f"Optimizer {temp} not recognized")

        temp = self.params['fb_optimizer']
        if temp.lower() == 'adam':
            self.fb_opt = Adam( self.params['fb_lr'] )
        elif temp.lower() == 'rmsprop':
            self.fb_opt = RMSprop( self.params['fb_lr'] )
        else:
            raise ValueError(f"fb_Optimizer {temp} not recognized")

    def Create_Network(self):
        input_layer = Input(shape=self.params['layers'][0],
                            name = 'Input')
        x = Dense(units=self.params['layers'][1],
                  activation=self.params['activation'],
                  kernel_initializer=self.initializer,
                  kernel_regularizer=self.regularizer,
                  name='Dense_1')(input_layer)
        for layer in range(2, len(self.params['layers'])-1):
            x = Dense(units=self.params['layers'][layer],
                      activation=self.params['activation'],
                      kernel_initializer=self.initializer,
                      kernel_regularizer=self.regularizer,
                      name=f'Dense_{layer}')(x)
        output = Dense(units=self.params['layers'][-1],
                       activation=self.params['output_act'],
                       kernel_initializer=self.initializer,
                       kernel_regularizer=self.regularizer,
                       name='Output')(x)
        self.mdl = Model(inputs=input_layer, outputs=output)

    def Create_FB_Network(self):
        input_layer = Input(shape=self.params['fb_layers'][0],
                            name = 'Input')
        x = Dense(units=self.params['fb_layers'][1],
                  activation=self.params['fb_activation'],
                  kernel_initializer=self.initializer,
                  kernel_regularizer=self.regularizer,
                  name='Dense_1')(input_layer)
        for layer in range(2, len(self.params['fb_layers'])-1):
            x = Dense(units=self.params['fb_layers'][layer],
                      activation=self.params['fb_activation'],
                      kernel_initializer=self.initializer,
                      kernel_regularizer=self.regularizer,
                      name=f'Dense_{layer}')(x)
        output = Dense(units=self.params['fb_layers'][-1],
                       activation=self.params['fb_output_act'],
                       kernel_initializer=self.initializer,
                       kernel_regularizer=self.regularizer,
                       name='Output')(x)
        self.fb = Model(inputs=input_layer, outputs=output)

    def Sample_Points(self):
        #According to the selected method, sample collocation points
        method = self.params['sample_method']
        if method == 'latin':
            from pyDOE import lhs
            cps = self.lb + (self.ub - self.lb)*lhs(self.ub.shape[0], self.N_f)
        elif method == 'uniform':
            cps = np.random.uniform(0, 1, size=(self.N_f, self.ub.shape[0]))
            cps = self.lb + (self.ub - self.lb)*cps
        elif method == 'sobol':
            import sobol
            cps = sobol.sample(dimension=self.ub.shape[0], n_points=self.N_f)
            cps = self.lb + (self.ub - self.lb)*cps
        else:
            raise ValueError(f'Sampling method {method} not recognized')
        #Return collocation points as tf tensors
        self.x_f_total = tf.cast(tf.Variable(cps[:, :-1], trainable=False),
                           self.DTYPE)
        self.t_f_total = tf.cast(tf.Variable(cps[:, -1:], trainable=False),
                           self.DTYPE)

    @tf.function
    def train_step(self):
        with G_Tape(watch_accessed_variables=False) as mdl_tape:
            #Watch solution weights
            mdl_tape.watch(self.mdl.trainable_variables)
            #--------------- Compute Free Boundary losses
            with G_Tape(watch_accessed_variables=False) as fb_tape:
                #Watch free boundary weights
                fb_tape.watch(self.fb.trainable_variables)
                #Compute Initial Free Boundary Condition
                fb_init = self.fb(self.ibc_fb_cond['Initial'].x,
                                      training=True)
                fb_init_loss = tf.math.reduce_mean(
                    tf.math.square(fb_init-self.ibc_fb_cond['Initial'].y)
                    )
                #Compute Dirichlet Free Boundary Condition
                if self.ibc_fb_cond['Dirichlet'] != None:
                    #Compute Free Boundary values
                    s_values = self.fb(self.ibc_fb_cond['Dirichlet'].x,
                                       training=True)
                    fb_dc = self.mdl(tf.concat(
                        [s_values,
                         self.ibc_fb_cond['Dirichlet'].x], axis=1),
                         training=True)
                    fb_dir_target = tf.nn.relu(
                        tf.ones_like(
                            s_values[:,-1]
                            ) * K - tf.math.reduce_min(s_values, axis=1)
                    )
                    fb_dir_loss = tf.math.reduce_mean(
                        tf.math.square(fb_dc - fb_dir_target)
                        )
                else:
                    fb_dir_loss = 0
                #Compute Neumann Free Boundary Condition
                if self.ibc_fb_cond['Neumann'] != None:
                    s_values = self.fb(self.ibc_fb_cond['Neumann'].x,
                                       training=True)
                    with G_Tape(watch_accessed_variables=False) as neu_fb_tape:
                        neu_fb_tape.watch(s_values)
                        pinn_nc_fb = self.mdl(
                            tf.concat([s_values,
                                       self.ibc_fb_cond['Neumann'].x],
                                      axis=1),
                                      training=True)
                    pinn_nc_fb = neu_fb_tape.gradient(pinn_nc_fb,
                                                      s_values)
                    fb_neu_target = -tf.one_hot(
                        tf.math.argmin( s_values, axis=1),
                        depth=n_dim
                        )
                    fb_neu_loss = tf.math.reduce_mean(
                        tf.math.square(pinn_nc_fb-fb_neu_target)
                        )
                else:
                    fb_neu_loss = 0
                #Compute final loss
                fb_loss = self.params['fb_weight'][0] * fb_init_loss +\
                self.params['fb_weight'][1] * fb_dir_loss +\
                self.params['fb_weight'][2] * fb_neu_loss
            #Compute gradient and apply optimizers for free boundary
            gradient_fb = fb_tape.gradient(fb_loss,
                                          self.fb.trainable_variables)
            self.fb_opt.apply_gradients( zip(gradient_fb,
                                          self.fb.trainable_variables) )

            #--------------- Compute PINN losses
            #Compute unsupervised loss
            s_values = self.fb(tf.concat([self.t_f_total], axis=-1),
                               training=False)
            temp = tf.reduce_sum(tf.cast(self.x_f_total < s_values,
                                         dtype=self.DTYPE),
                                axis=-1) < tf.ones(self.t_f_total.shape[0])
            x_f = self.x_f_total[ temp ]
            t_f = tf.reshape(self.t_f_total[ temp ], (-1,1) )
            variables = list()
            derivatives = list()
            for i in range(x_f.shape[1]):
                variables.append( x_f[:, i:i+1] )
            variables.append( t_f )
            with G_Tape(persistent=True,
                        watch_accessed_variables=False) as pinn_tape:
                #Watch independet variables
                for var in variables:
                    pinn_tape.watch( var )
                pinn_tape.watch(t_f)
                u_val = self.mdl(tf.concat(variables, axis=1),
                                    training=True)
                u_x = pinn_tape.gradient(u_val, x_f)
                for i, var in enumerate(variables[:-1]):
                    derivatives.append( pinn_tape.gradient(u_val, var) )
            unsup_loss = tf.reduce_mean(tf.square(
                self.pde(pinn_tape, variables, u_val, derivatives) ))
            #Compute Initial Condition
            pinn_init = self.mdl(self.ibc_cond['Initial'].x,
                                training=True)
            if len(pinn_init.shape) > 1:
                init_loss = tf.math.reduce_mean(
                    tf.math.square(pinn_init -\
                                  tf.expand_dims(self.ibc_cond['Initial'].y,
                                                 axis=-1 ) )
                    )
            else:
                init_loss = tf.math.reduce_mean(
                    tf.math.square(pinn_init -\
                                  self.ibc_cond['Initial'].y )
                    )
            #Compute Dirichlet Boundary Condition
            if self.ibc_cond['Dirichlet'] != None:
                pinn_dc = self.mdl(self.ibc_cond['Dirichlet'].x,
                                    training=True)
                if len(pinn_dc.shape) > 1:
                    dir_loss = tf.math.reduce_mean(
                        tf.math.square(pinn_dc -\
                                      tf.expand_dims(self.ibc_cond['Dirichlet'].y,
                                                     axis=-1 ) )
                        )
                else:
                    dir_loss = tf.math.reduce_mean(
                        tf.math.square(pinn_dc -\
                                      self.ibc_cond['Dirichlet'].y )
                        )
            else:
                dir_loss = 0
            #Compute Neumann Boundary Condition
            if self.ibc_cond['Neumann'] != None:
                with G_Tape(watch_accessed_variables=False) as neu_tape:
                    neu_tape.watch(self.ibc_cond['Neumann'].x)
                    pinn_nc = self.mdl(tf.concat([self.ibc_cond['Neumann'].x,
                                                  self.ibc_cond['Neumann'].t],
                                                axis=1),
                                        training=True)
                pinn_nc = neu_tape.gradient(pinn_nc, self.ibc_cond['Neumann'].x)
                neu_loss = tf.math.reduce_mean(
                    tf.math.square(pinn_nc-self.ibc_cond['Neumann'].y)
                    )
            else:
                neu_loss = 0

            #--------------- Compute total loss
            pinn_loss = (self.params['pde_weight'] * unsup_loss) +\
            self.params['sup_weight'][0] * init_loss +\
            self.params['sup_weight'][1] * dir_loss +\
            self.params['sup_weight'][2] * neu_loss +\
            self.params['fb_weight'][1] * fb_dir_loss +\
            self.params['fb_weight'][2] * fb_neu_loss
        #Compute gradient and apply optimizers
        gradient = mdl_tape.gradient(pinn_loss,self.mdl.trainable_variables)
        self.opt.apply_gradients(zip(gradient,self.mdl.trainable_variables))
        #Return losses
        final_fb_loss = fb_init_loss + fb_dir_loss + fb_neu_loss
        final_pinn_loss = unsup_loss + init_loss +\
         dir_loss + neu_loss + fb_dir_loss + fb_neu_loss
        return (unsup_loss, init_loss, dir_loss, neu_loss, fb_init_loss,
                fb_dir_loss, fb_neu_loss, final_fb_loss, final_pinn_loss)

    @tf.function
    def train_mdl_solo(self):
        with G_Tape(watch_accessed_variables=False) as mdl_tape:
            #Watch solution weights
            mdl_tape.watch(self.mdl.trainable_variables)
            #--------------- Compute Free Boundary losses
            #Compute Initial Free Boundary Condition
            fb_init = self.fb(self.ibc_fb_cond['Initial'].x,
                                  training=True)
            fb_init_loss = tf.math.reduce_mean(
                tf.math.square(fb_init-self.ibc_fb_cond['Initial'].y)
                )
            #Compute Dirichlet Free Boundary Condition
            if self.ibc_fb_cond['Dirichlet'] != None:
                #Compute Free Boundary values
                s_values = self.fb(self.ibc_fb_cond['Dirichlet'].x,
                                    training=True)
                fb_dc = self.mdl(tf.concat(
                    [s_values,
                      self.ibc_fb_cond['Dirichlet'].x], axis=1),
                      training=True)
                fb_dir_target = tf.nn.relu(
                    tf.ones_like(
                        s_values[:,-1]
                        ) * K - tf.math.reduce_min(s_values, axis=1)
                )
                fb_dir_loss = tf.math.reduce_mean(
                    tf.math.square(fb_dc - fb_dir_target)
                    )
            else:
                fb_dir_loss = 0
            #Compute Neumann Free Boundary Condition
            if self.ibc_fb_cond['Neumann'] != None:
                s_values = self.fb(self.ibc_fb_cond['Neumann'].x,
                                    training=True)
                with G_Tape(watch_accessed_variables=False) as neu_fb_tape:
                    neu_fb_tape.watch(s_values)
                    pinn_nc_fb = self.mdl(
                        tf.concat([s_values,
                                    self.ibc_fb_cond['Neumann'].x],
                                  axis=1),
                                  training=True)
                pinn_nc_fb = neu_fb_tape.gradient(pinn_nc_fb,
                                                  s_values)
                fb_neu_target = -tf.one_hot(
                    tf.math.argmin( s_values, axis=1),
                    depth=n_dim
                    )
                fb_neu_loss = tf.math.reduce_mean(
                    tf.math.square(pinn_nc_fb-fb_neu_target)
                    )
            else:
                fb_neu_loss = 0
            #Compute final loss
            fb_loss = self.params['fb_weight'][0] * fb_init_loss +\
            self.params['fb_weight'][1] * fb_dir_loss +\
            self.params['fb_weight'][2] * fb_neu_loss

            #--------------- Compute PINN losses
            #Compute unsupervised loss
            s_values = self.fb(tf.concat([self.t_f_total], axis=-1),
                               training=False)
            temp = tf.reduce_sum(tf.cast(self.x_f_total < s_values,
                                         dtype=self.DTYPE),
                                axis=-1) < tf.ones(self.t_f_total.shape[0])
            x_f = self.x_f_total[ temp ]
            t_f = tf.reshape(self.t_f_total[ temp ], (-1,1) )
            variables = list()
            derivatives = list()
            for i in range(x_f.shape[1]):
                variables.append( x_f[:, i:i+1] )
            variables.append( t_f )
            with G_Tape(persistent=True,
                        watch_accessed_variables=False) as pinn_tape:
                #Watch independet variables
                for var in variables:
                    pinn_tape.watch( var )
                pinn_tape.watch(t_f)
                u_val = self.mdl(tf.concat(variables, axis=1),
                                    training=True)
                u_x = pinn_tape.gradient(u_val, x_f)
                for i, var in enumerate(variables[:-1]):
                    derivatives.append( pinn_tape.gradient(u_val, var) )
            unsup_loss = tf.reduce_mean(tf.square(
                self.pde(pinn_tape, variables, u_val, derivatives) ))
            #Compute Initial Condition
            pinn_init = self.mdl(self.ibc_cond['Initial'].x,
                                training=True)
            if len(pinn_init.shape) > 1:
                init_loss = tf.math.reduce_mean(
                    tf.math.square(pinn_init -\
                                  tf.expand_dims(self.ibc_cond['Initial'].y,
                                                 axis=-1 ) )
                    )
            else:
                init_loss = tf.math.reduce_mean(
                    tf.math.square(pinn_init -\
                                  self.ibc_cond['Initial'].y )
                    )
            #Compute Dirichlet Boundary Condition
            if self.ibc_cond['Dirichlet'] != None:
                pinn_dc = self.mdl(self.ibc_cond['Dirichlet'].x,
                                    training=True)
                if len(pinn_dc.shape) > 1:
                    dir_loss = tf.math.reduce_mean(
                        tf.math.square(pinn_dc -\
                                      tf.expand_dims(self.ibc_cond['Dirichlet'].y,
                                                     axis=-1 ) )
                        )
                else:
                    dir_loss = tf.math.reduce_mean(
                        tf.math.square(pinn_dc -\
                                      self.ibc_cond['Dirichlet'].y )
                        )
            else:
                dir_loss = 0
            #Compute Neumann Boundary Condition
            if self.ibc_cond['Neumann'] != None:
                with G_Tape(watch_accessed_variables=False) as neu_tape:
                    neu_tape.watch(self.ibc_cond['Neumann'].x)
                    pinn_nc = self.mdl(tf.concat([self.ibc_cond['Neumann'].x,
                                                  self.ibc_cond['Neumann'].t],
                                                axis=1),
                                        training=True)
                pinn_nc = neu_tape.gradient(pinn_nc, self.ibc_cond['Neumann'].x)
                neu_loss = tf.math.reduce_mean(
                    tf.math.square(pinn_nc-self.ibc_cond['Neumann'].y)
                    )
            else:
                neu_loss = 0

            #--------------- Compute total loss
            pinn_loss = (self.params['pde_weight'] * unsup_loss) +\
            self.params['sup_weight'][0] * init_loss +\
            self.params['sup_weight'][1] * dir_loss +\
            self.params['sup_weight'][2] * neu_loss +\
            self.params['fb_weight'][1] * fb_dir_loss +\
            self.params['fb_weight'][2] * fb_neu_loss
        #Compute gradient and apply optimizers
        gradient = mdl_tape.gradient(pinn_loss,self.mdl.trainable_variables)
        self.opt.apply_gradients(zip(gradient,self.mdl.trainable_variables))
        #Return losses
        final_fb_loss = fb_init_loss + fb_dir_loss + fb_neu_loss
        final_pinn_loss = unsup_loss + init_loss +\
         dir_loss + neu_loss + fb_dir_loss + fb_neu_loss
        return (unsup_loss, init_loss, dir_loss, neu_loss, fb_init_loss,
                fb_dir_loss, fb_neu_loss, final_fb_loss, final_pinn_loss)

    @tf.function
    def train_fb_solo(self):
        #--------------- Compute Free Boundary losses
        with G_Tape(watch_accessed_variables=False) as fb_tape:
            #Watch free boundary weights
            fb_tape.watch(self.fb.trainable_variables)
            #Compute Initial Free Boundary Condition
            fb_init = self.fb(self.ibc_fb_cond['Initial'].x,
                                  training=True)
            fb_init_loss = tf.math.reduce_mean(
                tf.math.square(fb_init-self.ibc_fb_cond['Initial'].y)
                )
            #Compute Dirichlet Free Boundary Condition
            if self.ibc_fb_cond['Dirichlet'] != None:
                #Compute Free Boundary values
                s_values = self.fb(self.ibc_fb_cond['Dirichlet'].x,
                                    training=True)
                fb_dc = self.mdl(tf.concat(
                    [s_values,
                      self.ibc_fb_cond['Dirichlet'].x], axis=1),
                      training=True)
                fb_dir_target = tf.nn.relu(
                    tf.ones_like(
                        s_values[:,-1]
                        ) * K - tf.math.reduce_min(s_values, axis=1)
                )
                fb_dir_loss = tf.math.reduce_mean(
                    tf.math.square(fb_dc - fb_dir_target)
                    )
            else:
                fb_dir_loss = 0
            #Compute Neumann Free Boundary Condition
            if self.ibc_fb_cond['Neumann'] != None:
                s_values = self.fb(self.ibc_fb_cond['Neumann'].x,
                                    training=True)
                with G_Tape(watch_accessed_variables=False) as neu_fb_tape:
                    neu_fb_tape.watch(s_values)
                    pinn_nc_fb = self.mdl(
                        tf.concat([s_values,
                                    self.ibc_fb_cond['Neumann'].x],
                                  axis=1),
                                  training=True)
                pinn_nc_fb = neu_fb_tape.gradient(pinn_nc_fb,
                                                  s_values)
                fb_neu_target = -tf.one_hot(
                    tf.math.argmin( s_values, axis=1),
                    depth=n_dim
                    )
                fb_neu_loss = tf.math.reduce_mean(
                    tf.math.square(pinn_nc_fb-fb_neu_target)
                    )
            else:
                fb_neu_loss = 0
            #Compute final loss
            fb_loss = self.params['fb_weight'][0] * fb_init_loss +\
            self.params['fb_weight'][1] * fb_dir_loss +\
            self.params['fb_weight'][2] * fb_neu_loss
        #Compute gradient and apply optimizers for free boundary
        gradient_fb = fb_tape.gradient(fb_loss,
                                      self.fb.trainable_variables)
        self.fb_opt.apply_gradients( zip(gradient_fb,
                                      self.fb.trainable_variables) )
        #Return losses
        final_fb_loss = fb_init_loss + fb_dir_loss + fb_neu_loss
        return fb_init_loss, fb_dir_loss, fb_neu_loss, final_fb_loss

    def fit(self, wandb_run=None):
        #Early warning initialization
        self.early_warning = {'Target':np.inf,
                              'n_steps':0,
                              'top_mdl':None,
                              'weights':None}
        old_top_mdl, old_weights = None, None
        #Training
        self.u_losses, self.i_losses = list(), list()
        self.d_losses, self.n_losses = list(), list()
        self.b_i_losses = list()
        self.b_d_losses, self.b_n_losses = list(), list()
        self.b_losses, self.p_losses = list(), list()
        print_base = "{:<10}{:<15}{:<15}{:<15}{:<15}{:<15}{:<15}{:<15}{:<15}{:<15}"
        if self.params['fb_freezing'] == None:
            for epoch in tqdm(range(self.params['epochs']),
                              desc='PINNs - Training'):
                if epoch == 0:
                    print(print_base.format('Epoch', 'Unsupervised', 'Initial',
                                            'Dirichlet', 'Neumann',
                                            'FB_Init', 'FB_Dir', 'FB_Neu',
                                            'Free Boundary', 'Total'))
                    print('\n')
                #Case 1: more mdl steps for a single fb step
                for _ in range(self.params['steps_fb_per_pde'] -1):
                    self.train_mdl_solo();
                #Case 2: more fb steps for a single mdl step
                for _ in range(0, self.params['steps_fb_per_pde'] +1, -1):
                    self.train_fb_solo();
                u_l, i_l, d_l, n_l, b_i_l, b_d_l, b_n_l, b_l, p_l =\
                self.train_step()
                u_l, i_l = np.array(u_l), np.array(i_l)
                d_l, n_l = np.array(d_l), np.array(n_l)
                b_i_l = np.array(b_i_l)
                b_d_l, b_n_l = np.array(b_d_l), np.array(b_n_l)
                b_l, p_l = np.array(b_l), np.array(p_l)
                if epoch % self.params['verbose'] == 0:
                    print(print_base.format(epoch+1, format(u_l, '.20f')[:10],
                                            format(i_l, '.20f')[:10],
                                            format(d_l, '.20f')[:10],
                                            format(n_l, '.20f')[:10],
                                            format(b_i_l, '.20f')[:10],
                                            format(b_d_l, '.20f')[:10],
                                            format(b_n_l, '.20f')[:10],
                                            format(b_l, '.20f')[:10],
                                            format(p_l, '.20f')[:10]))
                self.u_losses.append(u_l)
                self.i_losses.append(i_l)
                self.d_losses.append(d_l)
                self.n_losses.append(n_l)
                self.b_i_losses.append(b_i_l)
                self.b_d_losses.append(b_d_l)
                self.b_n_losses.append(b_d_l)
                self.b_losses.append(b_l)
                self.p_losses.append(p_l)
                #Eventually, update wandb
                if wandb_run != None:
                    wandb_run.log({'Unsupervised':u_l,
                                  'Supervised':i_l + d_l + n_l,
                                  'Total':p_l,
                                   'Free Boundary':b_l,
                                   'Initial':i_l,
                                   'Dirichlet':d_l,
                                   'Neumann':n_l,
                                   'FB_Init':b_i_l,
                                   'FB_Dir':b_d_l,
                                   'FB_Neu':b_n_l})
                #Check for the early warning
                if p_l <= self.early_warning['Target']:
                    self.early_warning['Target'] = p_l
                    self.early_warning['n_steps'] = 0
                    self.early_warning['top_mdl'] = old_top_mdl
                    self.early_warning['weights'] = old_weights
                else:
                    self.early_warning['n_steps'] += 1
                    if self.early_warning['n_steps'] >= self.params['patience']:
                        break
                #Save model and weights for next step early warning
                old_top_mdl = tf.keras.models.clone_model( self.mdl )
                old_weights = self.mdl.get_weights()
        else:
            #Before freezing, both mdl and fb are training
            for epoch in tqdm(range(self.params['fb_freezing']),
                              desc='PINNs - Training'):
                if epoch == 0:
                    print(print_base.format('Epoch', 'Unsupervised', 'Initial',
                                            'Dirichlet', 'Neumann',
                                            'FB_Init', 'FB_Dir', 'FB_Neu',
                                            'Free Boundary', 'Total'))
                    print('\n')
                #Case 1: more mdl steps for a single fb step
                for _ in range(self.params['steps_fb_per_pde'] -1):
                    self.train_mdl_solo();
                #Case 2: more fb steps for a single mdl step
                for _ in range(0, self.params['steps_fb_per_pde'] +1, -1):
                    self.train_fb_solo();
                u_l, i_l, d_l, n_l, b_i_l, b_d_l, b_n_l, b_l, p_l =\
                self.train_step()
                u_l, i_l = np.array(u_l), np.array(i_l)
                d_l, n_l = np.array(d_l), np.array(n_l)
                b_i_l = np.array(b_i_l)
                b_d_l, b_n_l = np.array(b_d_l), np.array(b_n_l)
                b_l, p_l = np.array(b_l), np.array(p_l)
                if epoch % self.params['verbose'] == 0:
                    print(print_base.format(epoch+1, format(u_l, '.20f')[:10],
                                            format(i_l, '.20f')[:10],
                                            format(d_l, '.20f')[:10],
                                            format(n_l, '.20f')[:10],
                                            format(b_i_l, '.20f')[:10],
                                            format(b_d_l, '.20f')[:10],
                                            format(b_n_l, '.20f')[:10],
                                            format(b_l, '.20f')[:10],
                                            format(p_l, '.20f')[:10]))
                self.u_losses.append(u_l)
                self.i_losses.append(i_l)
                self.d_losses.append(d_l)
                self.n_losses.append(n_l)
                self.b_i_losses.append(b_i_l)
                self.b_d_losses.append(b_d_l)
                self.b_n_losses.append(b_d_l)
                self.b_losses.append(b_l)
                self.p_losses.append(p_l)
                #Eventually, update wandb
                if wandb_run != None:
                    wandb_run.log({'Unsupervised':u_l,
                                  'Supervised':i_l + d_l + n_l,
                                  'Total':p_l,
                                   'Free Boundary':b_l,
                                   'Initial':i_l,
                                   'Dirichlet':d_l,
                                   'Neumann':n_l,
                                   'FB_Init':b_i_l,
                                   'FB_Dir':b_d_l,
                                   'FB_Neu':b_n_l})
                #Check for the early warning
                if p_l <= self.early_warning['Target']:
                    self.early_warning['Target'] = p_l
                    self.early_warning['n_steps'] = 0
                    self.early_warning['top_mdl'] = old_top_mdl
                    self.early_warning['weights'] = old_weights
                else:
                    self.early_warning['n_steps'] += 1
                    if self.early_warning['n_steps'] >= self.params['patience']:
                        break
                #Save model and weights for next step early warning
                old_top_mdl = tf.keras.models.clone_model( self.mdl )
                old_weights = self.mdl.get_weights()
            #Now, freeze fb and train only mdl
            for epoch in tqdm(range(self.params['fb_freezing'],
                                    self.params['epochs']),
                              desc='PINNs - Training; Free Boundary Fixed'):
                u_l, i_l, d_l, n_l, b_i_l, b_d_l, b_n_l, b_l, p_l =\
                self.train_mdl_solo()
                u_l, i_l = np.array(u_l), np.array(i_l)
                d_l, n_l = np.array(d_l), np.array(n_l)
                b_i_l = np.array(b_i_l)
                b_d_l, b_n_l = np.array(b_d_l), np.array(b_n_l)
                b_l, p_l = np.array(b_l), np.array(p_l)
                if epoch % self.params['verbose'] == 0:
                    print(print_base.format(epoch+1, format(u_l, '.20f')[:10],
                                            format(i_l, '.20f')[:10],
                                            format(d_l, '.20f')[:10],
                                            format(n_l, '.20f')[:10],
                                            format(b_i_l, '.20f')[:10],
                                            format(b_d_l, '.20f')[:10],
                                            format(b_n_l, '.20f')[:10],
                                            format(b_l, '.20f')[:10],
                                            format(p_l, '.20f')[:10]))
                self.u_losses.append(u_l)
                self.i_losses.append(i_l)
                self.d_losses.append(d_l)
                self.n_losses.append(n_l)
                self.b_i_losses.append(b_i_l)
                self.b_d_losses.append(b_d_l)
                self.b_n_losses.append(b_d_l)
                self.b_losses.append(b_l)
                self.p_losses.append(p_l)
                #Eventually, update wandb
                if wandb_run != None:
                    wandb_run.log({'Unsupervised':u_l,
                                  'Supervised':i_l + d_l + n_l,
                                  'Total':p_l,
                                   'Free Boundary':b_l,
                                   'Initial':i_l,
                                   'Dirichlet':d_l,
                                   'Neumann':n_l,
                                   'FB_Init':b_i_l,
                                   'FB_Dir':b_d_l,
                                   'FB_Neu':b_n_l})
                #Check for the early warning
                if p_l <= self.early_warning['Target']:
                    self.early_warning['Target'] = p_l
                    self.early_warning['n_steps'] = 0
                    self.early_warning['top_mdl'] = old_top_mdl
                    self.early_warning['weights'] = old_weights
                else:
                    self.early_warning['n_steps'] += 1
                    if self.early_warning['n_steps'] >= self.params['patience']:
                        break
                #Save model and weights for next step early warning
                old_top_mdl = tf.keras.models.clone_model( self.mdl )
                old_weights = self.mdl.get_weights()
        #Recover information about optimal epoch in early warning
        self.mdl = tf.keras.models.clone_model( self.early_warning['top_mdl'] )
        self.mdl.set_weights(self.early_warning['weights'])
        top_epoch = epoch+1 - self.early_warning["n_steps"]
        print(f'Best loss achieved at step {top_epoch}')

    def plot_losses(self):
        plt.figure( figsize=(12,8) )
        plt.semilogy(self.u_losses, label='Unsupervised')
        plt.semilogy(np.array(self.i_losses) +\
                     np.array(self.d_losses) +\
                     np.array(self.n_losses),
                     label='Supervised')
        plt.semilogy(self.b_losses, label='Free Boundary')
        plt.semilogy(self.p_losses, label='PINN')
        plt.legend()
        plt.title('Losses')
        plt.show()

    def plot_unsupervised_test(self, test_sampler, test_ibc_cond,
                               test_ibc_fb_cond, to_print=True, output=False):
        #--------------- Compute Free Boundary losses
        #Compute Initial Free Boundary Condition
        if test_ibc_fb_cond['Initial'] != None:
            fb_init = self.fb(test_ibc_fb_cond['Initial'].x,
                                  training=False)
            fb_init_loss = tf.math.reduce_mean(
                tf.math.square(fb_init-test_ibc_fb_cond['Initial'].y)
                )
        else:
            fb_init_loss = 0
        #Compute Dirichlet Free Boundary Condition
        if test_ibc_fb_cond['Dirichlet'] != None:
            #Compute Free Boundary values
            s_values = self.fb(test_ibc_fb_cond['Dirichlet'].x,
                                training=True)
            fb_dc = self.mdl(tf.concat(
                [s_values,
                  test_ibc_fb_cond['Dirichlet'].x], axis=1),
                  training=True)
            fb_dir_target = tf.nn.relu(
                tf.ones_like(
                    s_values[:,-1]
                    ) * K - tf.math.reduce_min(s_values, axis=1)
            )
            fb_dir_loss = tf.math.reduce_mean(
                tf.math.square(fb_dc - fb_dir_target)
                )
        else:
            fb_dir_loss = 0
        #Compute Neumann Free Boundary Condition
        if test_ibc_fb_cond['Neumann'] != None:
            s_values = self.fb(test_ibc_fb_cond['Neumann'].x,
                                training=True)
            with G_Tape(watch_accessed_variables=False) as neu_fb_tape:
                neu_fb_tape.watch(s_values)
                pinn_nc_fb = self.mdl(
                    tf.concat([s_values,
                                test_ibc_fb_cond['Neumann'].x],
                              axis=1),
                              training=True)
            pinn_nc_fb = neu_fb_tape.gradient(pinn_nc_fb,
                                              s_values)
            fb_neu_target = -tf.one_hot(
                tf.math.argmin( s_values, axis=1),
                depth=n_dim
                )
            fb_neu_loss = tf.math.reduce_mean(
                tf.math.square(pinn_nc_fb-fb_neu_target)
                )
        else:
            fb_neu_loss = 0
        #Compute final loss
        fb_loss = fb_init_loss + fb_dir_loss + fb_neu_loss

        #--------------- Compute PINN losses
        #Compute unsupervised loss
        s_values = self.fb(tf.concat([test_sampler.t], axis=-1),
                            training=False)
        temp = tf.reduce_sum(tf.cast(test_sampler.x < s_values,
                                      dtype=self.DTYPE),
                            axis=-1) < tf.ones(test_sampler.t.shape[0])
        x_f = test_sampler.x[ temp ]
        t_f = tf.reshape(test_sampler.t[ temp ], (-1,1) )
        variables = list()
        derivatives = list()
        for i in range(x_f.shape[1]):
            variables.append( x_f[:, i:i+1] )
        variables.append( t_f )
        with G_Tape(persistent=True,
                    watch_accessed_variables=False) as pinn_tape:
            #Watch independet variables
            for var in variables:
                pinn_tape.watch( var )
            pinn_tape.watch(t_f)
            u_val = self.mdl(tf.concat(variables, axis=1),
                                training=True)
            u_x = pinn_tape.gradient(u_val, x_f)
            for i, var in enumerate(variables[:-1]):
                derivatives.append( pinn_tape.gradient(u_val, var) )
        unsup_loss = tf.reduce_mean(tf.square(
            self.pde(pinn_tape, variables, u_val, derivatives) ))
        #Compute Initial Condition
        if test_ibc_cond['Initial'] != None:
            pinn_init = self.mdl(test_ibc_cond['Initial'].x,
                                  training=False)
            if len(pinn_init.shape) > 1:
                init_loss = tf.math.reduce_mean(
                    tf.math.square(pinn_init -\
                                  tf.expand_dims(test_ibc_cond['Initial'].y,
                                                 axis=-1 ) )
                    )
            else:
                init_loss = tf.math.reduce_mean(
                    tf.math.square(pinn_init -\
                                  test_ibc_cond['Initial'].y )
                    )
        else:
            init_loss = 0
        #Compute Dirichlet Boundary Condition
        if test_ibc_cond['Dirichlet'] != None:
            pinn_dc = self.mdl(test_ibc_cond['Dirichlet'].x,
                                training=True)
            if len(pinn_dc.shape) > 1:
                dir_loss = tf.math.reduce_mean(
                    tf.math.square(pinn_dc -\
                                  tf.expand_dims(test_ibc_cond['Dirichlet'].y,
                                                 axis=-1 ) )
                    )
            else:
                dir_loss = tf.math.reduce_mean(
                    tf.math.square(pinn_dc -\
                                  test_ibc_cond['Dirichlet'].y )
                    )
        else:
            dir_loss = 0
        #Compute Neumann Boundary Condition
        if test_ibc_cond['Neumann'] != None:
            with G_Tape(watch_accessed_variables=False) as neu_tape:
                neu_tape.watch(test_ibc_cond['Neumann'].x)
                pinn_nc = self.mdl(tf.concat([test_ibc_cond['Neumann'].x,
                                              test_ibc_cond['Neumann'].t],
                                            axis=1),
                                    training=False)
            pinn_nc = neu_tape.gradient(pinn_nc, test_ibc_cond['Neumann'].x)
            neu_loss = tf.math.reduce_mean(
                tf.math.square(pinn_nc-test_ibc_cond['Neumann'].y)
                )
        else:
            neu_loss = 0

        #--------------- Compute total loss
        pinn_loss = unsup_loss + init_loss + dir_loss +\
        neu_loss + fb_dir_loss + fb_neu_loss

        u_l, i_l = np.array(unsup_loss), np.array(init_loss)
        d_l, n_l = np.array(dir_loss), np.array(neu_loss)
        b_i_l = np.array(fb_init_loss)
        b_d_l, b_n_l = np.array(fb_dir_loss), np.array(fb_neu_loss)
        b_l, p_l = np.array(fb_loss), np.array(pinn_loss)

        if to_print:
            print_base = "{:<10}{:<15}{:<15}{:<15}{:<15}{:<15}{:<15}{:<15}{:<15}{:<15}"
            print(print_base.format('', 'Unsupervised', 'Initial',
                                    'Dirichlet', 'Neumann', 'FB_Init', 'FB_Dir',
                                    'FB_Neu', 'Free Boundary', 'Total'))
            print(print_base.format('', format(u_l, '.20f')[:10],
                                    format(i_l, '.20f')[:10],
                                    format(d_l, '.20f')[:10],
                                    format(n_l, '.20f')[:10],
                                    format(b_i_l, '.20f')[:10],
                                    format(b_d_l, '.20f')[:10],
                                    format(b_n_l, '.20f')[:10],
                                    format(b_l, '.20f')[:10],
                                    format(p_l, '.20f')[:10]))
        if output:
            return u_l, i_l, d_l, n_l, b_i_l, b_d_l, b_n_l, b_l, p_l


### Example

In [9]:

from tensorflow.keras.optimizers.schedules import ExponentialDecay as ex_d
my_lr = ex_d(1e-2, 320, 0.975, staircase=False)
fb_lr = ex_d(1e-2, 80, 0.975, staircase=False)

params = {'sample_method':'sobol',
          'layers':[n_dim+1, 20, 20, 20, 20, 20, 20, 20, 20, 1],
          'activation':'tanh',
          'output_act':'linear', 'initializer':'glorot_normal',
          'fb_layers':[1, 100, 100, 100, n_dim], 'fb_activation':'tanh',
          'fb_output_act':'linear', 'fb_initializer':'glorot_normal',
          'lr':my_lr, 'optimizer':'rmsprop',
          'fb_lr':fb_lr, 'fb_optimizer':'rmsprop', 'steps_fb_per_pde':4,
          'pde_weight':1, 'epochs':15000, 'verbose':100}

my_pinn = FreeBoundary_PINN(params, pde, pinn_conditions,
                            fb_conditions, lb, ub, N_f=120000, DTYPE=DTYPE)

START = time.time()
my_pinn.fit()
train_time = time.time() - START
my_pinn.plot_losses()

START = time.time()
values = my_pinn.plot_unsupervised_test(test_sampler, pinn_cond_test,
                                        fb_cond_test, output=True)
test_time = time.time() - START


PINNs - Training:   0%|          | 0/15000 [00:00<?, ?it/s]

Epoch     Unsupervised   Initial        Dirichlet      Neumann        FB_Init        FB_Dir         FB_Neu         Free Boundary  Total          


1         0.00217668     12.6619749     14.0454292     0.00000000     96.2858352     29.5317935     0.37270227     126.190330     56.6140747     
101       0.06323869     1.06593942     0.90398234     0.00000000     0.56724703     0.01674182     0.30657500     0.89056384     2.35647726     


KeyboardInterrupt: 